In [1]:
import os
import requests

In [2]:
#pip install joblib

In [3]:
#pip install --upgrade pip

In [4]:
#pip install --upgrade pip

In [5]:
#pip install --upgrade pip

In [6]:
pip install kfp

  Using cached https://files.pythonhosted.org/packages/00/f7/4f196c55f1c2713d3edc8252c4b45326306eef4dc10048f13916fe446e2b/kubernetes-9.0.0-py2.py3-none-any.whl
  Found existing installation: kubernetes 10.0.1
    Uninstalling kubernetes-10.0.1:
ERROR: Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: 'LICENSE'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install subprocess.run --user

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install wget

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import wget
url = 'https://raw.githubusercontent.com/lobosor/CSV/master/kc_house_data.csv'
wget.download(url, 'data.csv')

'data (30).csv'

In [10]:
import kfp
import kfp.gcp as gcp
import kfp.dsl as dsl
import kfp.compiler as compiler
import kfp.components as comp

import kubernetes as k8s

In [11]:
#link = 'https://raw.githubusercontent.com/lobosor/CSV/master/kc_house_data.csv'
#file= requests.get(link)
#open('dataset.csv','wb').write(file.content)

In [12]:
#Define a Python function
# Advanced function
# Demonstrates imports, helper functions and multiple outputs
from typing import NamedTuple

def my_divmod(dividend: float, 
              divisor: float, 
              output_dir: str = './'
             ) -> NamedTuple('MyDivmodOutput', [('quotient', float), ('remainder', float)]):
    
    '''Divides two numbers and calculate  the quotient and remainder'''
    
    #Imports inside a component function:
    import numpy as np

    #This function demonstrates how to use nested functions inside a component function:
    def divmod_helper(dividend, divisor):
        return np.divmod(dividend, divisor)

    (quotient, remainder) = divmod_helper(dividend, divisor)

    from tensorflow.python.lib.io import file_io
    import json
    
    # Exports a sample tensorboard:
    metadata = {
      'outputs' : [{
        'type': 'tensorboard',
        'source': 'gs://ml-pipeline-dataset/tensorboard-train',
      }]
    }
    with open(output_dir + 'mlpipeline-ui-metadata.json', 'w') as f:
        json.dump(metadata, f)

    # Exports two sample metrics:
    metrics = {
      'metrics': [{
          'name': 'quotient',
          'numberValue':  float(quotient),
        },{
          'name': 'remainder',
          'numberValue':  float(remainder),
        }]}

    with file_io.FileIO(output_dir + 'mlpipeline-metrics.json', 'w') as f:
        json.dump(metrics, f)

    from collections import namedtuple
    divmod_output = namedtuple('MyDivmodOutput', ['quotient', 'remainder'])
    return divmod_output(quotient, remainder)
test_op=comp.func_to_container_op(my_divmod)

In [13]:
#Define a Python function
import requests

def DownloadDataset() :
    link = 'https://raw.githubusercontent.com/lobosor/CSV/master/kc_house_data.csv'
    import subprocess
    from subprocess import run
    run("pip3 install requests")

    import requests
    #from requests import get
   #'''Calculates sum of two arguments'''
   #return a + b
    #def getit(link):
    file = requests.get(link)
    return file.content
    #file=getit(link)
    #return file
    open('dataset.csv','wb').write(file.content)
def download_csv() -> str:
    url = 'https://raw.githubusercontent.com/lobosor/CSV/master/kc_house_data.csv'
    from subprocess import run
    run("pip3 install requests")
    import requests
    local_file = 'data.csv'
    r = requests.get(url)
    with open(local_file, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
    return local_file

def ddl():
    from subprocess import run
    run("pip3 install wget")
    import wget
    url = 'https://raw.githubusercontent.com/lobosor/CSV/master/kc_house_data.csv'
    wget.download(url, 'data.csv')

In [14]:
down_op = comp.func_to_container_op(ddl,base_image="tensorflow/tensorflow:1.15.0-py3")
down_op1 = comp.func_to_container_op(download_csv,base_image="python3:latest")
down_op2 = comp.func_to_container_op(DownloadDataset,base_image="sabbir1cse/ubuntu-python-pip-supervisor")

In [15]:
#train 
def train():
    #dataset='/tmp/WS'
    import pandas as pd
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.model_selection import train_test_split
    #import joblib
    #data_path = data
    #trained_model_path = os.getenv('TRAINED_MODEL_PATH')
    #if dataset is None:
    #    raise RuntimeError('Dataset must not be none')
    data = pd.read_csv(dataset)
    train_data = data.drop(['id', 'price'], axis=1)
    labels = data['price']
    x_train, x_test, y_train, y_test = train_test_split(train_data, labels, test_size=0.10, random_state=2)
    _clf = GradientBoostingRegressor(n_estimators=400, max_depth=5, min_samples_split=2,learning_rate=0.1, loss='ls')
    _clf.fit(x_train, y_train)

    #joblib.dump(_clf, trained_model_path)
    return _clf
    
train_op = comp.func_to_container_op(train,base_image="python:latest")

In [16]:
import kfp.dsl as dsl
@dsl.pipeline(
    name='Workshop pipeline',
    description='None'
)

def My_Pipeline():
    #link = 'https://raw.githubusercontent.com/lobosor/CSV/master/kc_house_data.csv'
    test_task =test_op(16,8)
    down_task =down_op()
    down_task =down_op1()
    down_task =down_op2()
    train_task = train_op()
    #predict_task = predict_op

In [17]:
compiler.Compiler().compile(My_Pipeline, "my-pipeline.zip")

In [18]:
client = kfp.Client()
my_experiment = client.create_experiment(name='demo')
my_run = client.run_pipeline(my_experiment.id, 'my-pipeline', 'my-pipeline.zip')